# Peter's Clustering Cities in Toronto 

<b> Code to import libraries </b>

In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

print('Libraries imported.')

Libraries imported.


In [11]:
from bs4 import BeautifulSoup
import requests

In [18]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

In [19]:
page = requests.get(url)
page

<Response [200]>

In [21]:
soup = BeautifulSoup(page.content, 'html.parser')

In [171]:
rows = soup.find_all('tr')
rowlength = len(rows) - 5 #because last few rows are styling

In [166]:
### define the dataframe columns
column_names = ['PostalCode', 'Borough', 'Neighborhood'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

In [167]:
for index in range (1, rowlength):
    PostalCode = rows[index].find_all('td')[0].get_text()
    Borough = rows[index].find_all('td')[1].get_text()
    Neighborhood = rows[index].find_all('td')[2].get_text().splitlines()[0]
    str(Neighborhood)
    
    neighborhoods = neighborhoods.append({'PostalCode': PostalCode,
                                          'Borough': Borough,
                                          'Neighborhood': Neighborhood}, ignore_index=True)




### 3. Cleaning up Data to drop row if borough is not assigned, make neighborhood borough if no neighborhood

In [180]:
#Drop Row if Borough is null
neighborhoods['Borough'] = neighborhoods['Borough'].replace('Not assigned', np.nan)
neighborhoods = neighborhoods.dropna(axis=0, subset=['Borough'])

In [184]:
#Change neighborhood to borough if neighborhood = not assigned
neighborhoods['Neighborhood'] = neighborhoods['Neighborhood'].replace('Not assigned', neighborhoods['Borough'])

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


#### Now I group by PostalCode and Borough and append the neighborhoods together

In [235]:
neighborhoods['Neighborhood'] = neighborhoods['Neighborhood'].astype(str)

In [236]:
result = neighborhoods.groupby(['PostalCode', 'Borough']).agg(','.join)

In [238]:
result

,,Neighborhood
PostalCode,Borough,
M1B,Scarborough,"Rouge,Malvern"
M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
M1E,Scarborough,"Guildwood,Morningside,West Hill"
M1G,Scarborough,Woburn
M1H,Scarborough,Cedarbrae
M1J,Scarborough,Scarborough Village
M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park"
M1L,Scarborough,"Clairlea,Golden Mile,Oakridge"
M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West"


In [239]:
result.shape

(103, 1)